<h1 style='font-size:40px'> tf.data Pipeline</h1>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            In this notebook I'll practice my skills with the tf.data module solving Exercise 9 from Hands-On Machine Learning with Scikit-Learn and TensorFlow's Chapter 13.
        </li>
        <li> 
            The Exercise commands us the following:
            <p style='font-style:italic;margin-top:10px'> 
                Load the Fashion MNIST dataset (introduced in Chapter 10); split
it into a training set, a validation set, and a test set; shuffle the
training set; and save each dataset to multiple TFRecord files.
Each record should be a serialized Example protobuf with two
features: the serialized image (use tf.io.serialize_tensor()
to serialize each image), and the label. 11 Then use tf.data to create
an efficient dataset for each set. Finally, use a Keras model to
train these datasets, including a preprocessing layer to standardize
each input feature. Try to make the input pipeline as efficient as
possible, using TensorBoard to visualize profiling data.
            </p>
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Data Importing & Splitting</h2>

In [1]:
# Loading the fashion_mnist dataset.
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Now, generating the validation set with `train_test_split`.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.1, random_state=42)

4422102/4422102 [==============================] - 0s 0us/step


In [2]:
# Storing each one of the sets in a tf.data.Dataset object.
# The classes store 1000 elements batches. The groups' data will be put into a .tfrecord file.
from tensorflow.data import Dataset
batch_size = 1000
train = Dataset.from_tensor_slices((X_train, y_train)).shuffle(54000).batch(batch_size)
val = Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)
test = Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

In [3]:
# We'll produce 1000 instances .tfrecord files. One corresponding to a bacth created.
train_files = int(len(X_train) / batch_size)
val_files = int(len(X_val) / batch_size)
test_files = int(len(X_test) / batch_size)

In [4]:
from tensorflow.train import BytesList, Example, Features, Feature, Int64List
from tensorflow.io import serialize_tensor
def create_example(image, target):
    serialized_image, serialized_target = serialize_tensor(image), serialize_tensor(target)
    example = Example(
        features=Features(
            feature={
            'pixels':Feature(bytes_list=BytesList(value=[serialized_image.numpy()])),
            'target':Feature(bytes_list=BytesList(value=[serialized_target.numpy()]))#Feature(int64_list=Int64List(value=[target]))
        }
        ))
    return example

for image, target in train.take(1):
    print(create_example(image, target))

features {
  feature {
    key: "pixels"
    value {
      bytes_list {
        value: "\010\004\022\r\022\003\010\350\007\022\002\010\034\022\002\010\034\"\200\355/\000\000\000\000\000\000\000\000\000\000\000\0001\223ud\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\001\000\000\000\205\244}\203\033\000\001\001\000\000\000\000\000\000\000\000\000\000\000\000\000\000\001\000\000\000\000H\254\250\276fP\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\021\'R\203\225\244\221\\oZ/\t\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000byi\\bf}{`dbgbR\013\000\000\000\000\000\000\000\000\000\000\000\000<\211m`^dbk\227Z`\\TVb^\000\000\000\000\000\000\000\000\000\000\000\000m\211gi``Zd\207T\\NNRD^\017\000\000\000\000\000\000\000\000\000\000\000{\215\221^V^V^{T\\HNLBX\027\000\000\000\000\000\000\000\000\000\000\000\211w\302\264P`Z\\sZdVB@RX)\000\000\000\000\000\000\000\000\000\000\001\223o\306\377`f\\Xobm\\8dfT:\000\000\000\000\00

<p style='color:red'> Função para .tfrecords 

<h2 style='font-size:30px'> </h2>